## Task: Random Forest Model for BBBP Classifier using RDkit Descriptors and Hyperparameter Optimization

* Develop a random forest model for the BBBP classifier using the default hyperparameters and rdkit descriptors, and report the performance measure (ROC_AUC) for all three datasets.

* List the default hyperparameters used in the model (n_estimators, max_depth, min_samples_leaf, min_impurity_decrease, max_features).

* Optimize the model using grid search to find the optimal set of hyperparameters and report the performance measure (ROC_AUC) for all three datasets with your optimized hyperparameters.

* Compare your results with the baseline model and the results reported in Table 4 of the following paper: https://jcheminf.biomedcentral.com/articles/10.1186/s13321-020-00479-8, and with the results of task 3.

### import modules

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from sklearn.model_selection import (train_test_split, 
                                     cross_val_predict, 
                                     GridSearchCV)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

### import data

In [44]:
bbbp = pd.read_csv("BBBP.csv")
bbbp.head()

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


# calculate RDkit descriptors

In [45]:
# suppress warnings from invalid molecules - why does this happen???
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [46]:
# function: generate canon SMILES
def gen_canon_smiles(smiles_list):
    
    invalid_ids = []
    canon_smiles = []

    for i in range(len(smiles_list)):   
        mol = Chem.MolFromSmiles(smiles_list[i])
        
        # do not append NoneType if invalid
        if mol is None: 
            invalid_ids.append(i)
            continue

        canon_smiles.append(Chem.MolToSmiles(mol))

    return canon_smiles, invalid_ids


# function: calculate molecular descriptors
def RDkit_descriptors(smiles):
    # convert smiles into their corresponding molecular graphs
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    
    # calculate all descriptors available in RDkit and save to variable
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
        
    return Mol_descriptors, desc_names 

In [47]:
# generate canon smiles
canon_smiles, invalid_ids = gen_canon_smiles(bbbp.smiles)

# drop rows with invalid SMILES
bbbp = bbbp.drop(invalid_ids)

# replace SMILES with canon SMILES
bbbp.smiles = canon_smiles

# drop duplicates to prevent train/valid/test contamination
bbbp.drop_duplicates(subset=['smiles'], inplace=True)

In [48]:
# function call
Mol_descriptors, desc_names = RDkit_descriptors(bbbp.smiles)

# put descriptors and descriptor names into dataframe
desc = pd.DataFrame(Mol_descriptors,columns=desc_names)
desc.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,8.297086,-4.334066,8.297086,0.000000,0.859050,294.802,273.634,294.126082,109,1,...,0,0,0,0,0,0,0,0,0,0
1,13.093184,-4.643037,13.093184,0.649388,0.474821,360.325,333.109,359.141884,130,0,...,0,0,0,0,0,0,0,0,0,0
2,16.373716,-4.214838,16.373716,0.240165,0.874740,361.373,341.213,361.143784,138,0,...,0,0,0,0,0,0,0,0,0,0
3,11.991523,-4.317357,11.991523,0.821900,0.784810,290.407,264.199,290.199428,116,0,...,0,0,0,0,0,0,0,0,0,0
4,13.928423,-3.966667,13.928423,0.341483,0.709265,435.889,417.745,435.065569,152,0,...,1,0,0,0,0,0,0,0,0,0


In [88]:
# add labels to df so it is easier to rm nans
desc['p_np'] = bbbp['p_np']
desc = desc[~desc.isin([np.nan, np.inf, -np.inf]).any(1)]

In [89]:
X = desc.drop(columns=['p_np']) # features
y = desc.p_np # labels

In [90]:
from sklearn.preprocessing import MinMaxScaler

# otherwise I get a ValueError
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

### split data

In [91]:
# split data into training set and everything else
X_train, X_valid_and_test, y_train, y_valid_and_test = train_test_split(X, y, train_size=0.8)

# split everything else into validation and test sets
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_and_test, y_valid_and_test, test_size=0.5)

### random forest model

In [92]:
# train and fit
forest = RandomForestClassifier().fit(X_train, y_train)

In [93]:
# compute decision scores
y_train_scores = forest.predict_proba(X_train)
y_valid_scores = forest.predict_proba(X_valid)
y_test_scores = forest.predict_proba(X_test)

# print ROC-AOC scores
print("train_set scores:", roc_auc_score(y_train, y_train_scores[:,1]))
print("valid_set scores:", roc_auc_score(y_valid, y_valid_scores[:,1]))
print("test_set scores :", roc_auc_score(y_test, y_test_scores[:,1]))

train_set scores: 0.9999471374332757
valid_set scores: 0.7695749440715883
test_set scores : 0.75589656152316


The model almost perfectly fits the training set. It performs much, much worse on the validation and test sets. The model has severely overfit the training set.

In [95]:
# print default hyperparams
forest.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [96]:
# optimize model with grid search - is it still necessary?
params = {
    'n_estimators': [int(x) for x in np.linspace(100, 800, 4)], 
    'max_features': ['sqrt'], 
    'min_samples_split': [int(x) for x in np.linspace(2, 6, 5)],
    'min_samples_leaf': [int(x) for x in np.linspace(1, 4, 4)]
}

forest_op = GridSearchCV(RandomForestClassifier(), params, cv=3)
forest_op.fit(X_train, y_train)
forest_op.best_estimator_

RandomForestClassifier(min_samples_leaf=2, min_samples_split=3)

In [100]:
forest_op = forest_op.best_estimator_

In [101]:
# compute decision scores
y_train_scores = forest_op.predict_proba(X_train)
y_valid_scores = forest_op.predict_proba(X_valid)
y_test_scores = forest_op.predict_proba(X_test)

# print ROC-AOC scores
print("train_set scores:", roc_auc_score(y_train, y_train_scores[:,1]))
print("valid_set scores:", roc_auc_score(y_valid, y_valid_scores[:,1]))
print("test_set scores :", roc_auc_score(y_test, y_test_scores[:,1]))

train_set scores: 0.999389143673408
valid_set scores: 0.7358458096713131
test_set scores : 0.7619352088661553


The results of my random forest model turned out to be worse than the baseline model of the paper. Both the baseline model in the paper and my model overfit the training set, however, my model performs much worse on the validation and test sets. The results of this model are also worse than my model from task 3, which had a similar performance to the model in the paper.